# Import EEGlab File into HyPyP

**Authors**: Ghazaleh Ranjbaran  
**Date**: 2021-12-16

## Overview

This notebook demonstrates how to import EEG data preprocessed in EEGlab into HyPyP. The workflow includes:

- **Loading required libraries** for data science and EEG analysis.
- **Loading the EEGlab dataset** for two participants using MNE's EEGlab import functions.
- **Equalizing the number of epochs** between participants (if needed) since epochs may differ in count but must have the same number of samples per epoch.
- **Renaming channels** based on the international 10/20 system. Channels not in the standard list (e.g., EOG channels) are removed.
- **Assigning a standard montage** to the data so that it is compatible with HyPyP visualization and statistical analysis tools.

The expected outcome is that the resulting epochs (named `epo_ref1` and `epo_ref2`) will be compatible with HyPyP tools. For more details on further processing, please refer to the [HyPyP getting started tutorial](https://github.com/ppsp-team/HyPyP/blob/master/tutorial/getting_started.ipynb).

## Load useful libs

### Data science

In [ ]:
import numpy as np

### MNE

In [ ]:
import mne

## Load Data

In this section, we load EEGlab data. For each participant, both the `.set` file and corresponding `.fdt` file (if needed) should be located in the same directory. The data is assumed to be epoched. Although participants can have different numbers of epochs, each epoch must contain the same number of samples.

In [ ]:
# Define file paths for the EEGlab datasets for participant 1 and 2
path_1 = "../data/EEGLAB/eeglab_data_epochs_ica.set"
path_2 = "../data/EEGLAB/eeglab_data_epochs_ica.set"

# Import EEGlab data for participant 1
epo1 = mne.io.read_epochs_eeglab(path_1)

# Import EEGlab data for participant 2
epo2 = mne.io.read_epochs_eeglab(path_2)

print('EEGlab data loaded for both participants.')

In [ ]:
# Equalize the number of epochs between the two participants
mne.epochs.equalize_epoch_counts([epo1, epo2])
print('Epoch counts equalized.')

## Rename Channels Using the International 10/20 System

To ensure compatibility with the MNE template and HyPyP tools, we select channels following the international 10/20 system. Note that channels related to EOG (eyes) are removed from the analysis.

In [ ]:
StanSys = ['Nz', 'Fp1', 'Fpz', 'Fp2', 'F7', 'F9', 'PO3',
           'F3', 'Fz', 'F4', 'F8', 'T3', 'C3', 'CP3', 'PO4',
           'Cz', 'C4', 'T4', 'T5', 'P3', 'Pz', 'P3', 'PO7',
           'P4', 'T6', 'O1', 'Oz', 'O2', 'Iz', 'P1', 'PO8',
           'AF3', 'AF7', 'AF4', 'AF8', 'F6', 'F10', 'F2',  'F5', 
           'FC1', 'FC3', 'FC5', 'FCz', 'FC2', 'FC4', 'FC6', 'F1',
           'FT9', 'FT7', 'FT8', 'FT10', 'C5', 'C6', 'CPz', 'CP1', 
           'CP5', 'CP2', 'CP4', 'CP6', 'TP8', 'TP10', 'TP7', 'TP9', 
           'P5', 'P7', 'P9', 'P2', 'P4', 'P6', 'P8', 'P10', 'POz']

print('Standard 10/20 channels defined.')

In [ ]:
# Create a lower-case version of the standard channel names for easier matching
low_StanSys = []
for name in StanSys:
    low_StanSys.append(name.lower())
print('Lower-case channel names created.')

In [ ]:
# Get the channel names from the EEGlab data for each participant
names_epo1 = np.array([ch['ch_name'] for ch in epo1.info['chs']])
names_epo2 = np.array([ch['ch_name'] for ch in epo2.info['chs']])
print('Channel names extracted from EEGlab data.')

In [ ]:
# Create a copy of participant 1's epochs for renaming channels
epo_ref1 = epo1.copy()

# Iterate over channels and rename those that match the standard list;
# drop channels not found in the standard 10/20 system
for idx in range(len(names_epo1)):
    aux_name = names_epo1[idx].lower()
    if aux_name in low_StanSys:
        ind = low_StanSys.index(aux_name)
        nw_ch = StanSys[ind]
        mne.rename_channels(epo_ref1.info, {names_epo1[idx]: nw_ch})
    else:
        epo_ref1.drop_channels(names_epo1[idx])

print('Participant 1 channels renamed and non-standard channels dropped.')

In [ ]:
# Create a copy of participant 2's epochs for renaming channels
epo_ref2 = epo2.copy()

# Iterate over channels for participant 2 and perform renaming or dropping
for idx in range(len(names_epo2)):
    aux_name = names_epo2[idx].lower()
    if aux_name in low_StanSys:
        ind = low_StanSys.index(aux_name)
        nw_ch = StanSys[ind]
        mne.rename_channels(epo_ref2.info, {names_epo2[idx]: nw_ch})
    else:
        epo_ref2.drop_channels(names_epo2[idx])

print('Participant 2 channels renamed and non-standard channels dropped.')

In [ ]:
# Create a standard montage using MNE's 'standard_1020' template
locations = mne.channels.make_standard_montage('standard_1020', head_size=0.092)
print('Standard montage created.')

In [ ]:
# Apply the standard montage to participant 2's data
epo_ref2.set_montage(locations)
print('Montage set for participant 2.')

In [ ]:
# Apply the standard montage to participant 1's data
epo_ref1.set_montage(locations)
print('Montage set for participant 1.')

epo_ref1 and epo_ref2 are now compatible with tools provided by HyPyP (for visualization and statictical analyses check [this](https://github.com/ppsp-team/HyPyP/blob/master/tutorial/getting_started.ipynb) tutorial)

## Final Outcome

The processed epochs `epo_ref1` and `epo_ref2` now have channels renamed according to the international 10/20 system and a standard montage applied. They are ready to be used with HyPyP's visualization and statistical analysis tools. For further instructions, please see the [HyPyP getting started tutorial](https://github.com/ppsp-team/HyPyP/blob/master/tutorial/getting_started.ipynb).